## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

### Data Load

In [2]:
import pandas as pd

# 데이터 로드
TRCMWS_train = pd.read_csv('../data/train_dataset_25_TRCMWS.csv')
TRCMWS_test = pd.read_csv('../data/test_dataset_25_TRCMWS.csv')

TRDQAZ_train = pd.read_csv('../data/train_dataset_25_TRDQAZ.csv')
TRDQAZ_test = pd.read_csv('../data/test_dataset_25_TRDQAZ.csv')

TRJXFG_train = pd.read_csv('../data/train_dataset_25_TRJXFG.csv')
TRJXFG_test = pd.read_csv('../data/test_dataset_25_TRJXFG.csv')

TRVNRY_train = pd.read_csv('../data/train_dataset_25_TRVNRY.csv')
TRVNRY_test = pd.read_csv('../data/test_dataset_25_TRVNRY.csv')

TRXQMD_train = pd.read_csv('../data/train_dataset_25_TRXQMD.csv')
TRXQMD_test = pd.read_csv('../data/test_dataset_25_TRXQMD.csv')

TRYBLT_train = pd.read_csv('../data/train_dataset_25_TRYBLT.csv')
TRYBLT_test = pd.read_csv('../data/test_dataset_25_TRYBLT.csv')

TRZKPL_train = pd.read_csv('../data/train_dataset_25_TRZKPL.csv')
TRZKPL_test = pd.read_csv('../data/test_dataset_25_TRZKPL.csv')

In [3]:
TRCMWS_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38089 entries, 0 to 38088
Data columns (total 90 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      38089 non-null  object 
 1   시술_당시_나이                38089 non-null  object 
 2   임신_시도_또는_마지막_임신_경과_연수   38089 non-null  object 
 3   시술_유형                   38089 non-null  object 
 4   배란_자극_여부                38089 non-null  int64  
 5   배란_유도_유형                38089 non-null  object 
 6   단일_배아_이식_여부             37257 non-null  float64
 7   착상_전_유전_검사_사용_여부        38089 non-null  float64
 8   착상_전_유전_진단_사용_여부        37257 non-null  float64
 9   남성_주_불임_원인              38089 non-null  int64  
 10  남성_부_불임_원인              38089 non-null  int64  
 11  여성_주_불임_원인              38089 non-null  int64  
 12  여성_부_불임_원인              38089 non-null  int64  
 13  부부_주_불임_원인              38089 non-null  int64  
 14  부부_부_불임_원인              38089 non-null

In [4]:
# ID 열을 제외한 특성과 타겟 변수 분리
TRCMWS_X = TRCMWS_train.drop(['임신_성공_여부', 'ID'], axis=1)
TRCMWS_y = TRCMWS_train['임신_성공_여부']

TRDQAZ_X = TRDQAZ_train.drop(['임신_성공_여부', 'ID'], axis=1)
TRDQAZ_y = TRDQAZ_train['임신_성공_여부']

TRJXFG_X = TRJXFG_train.drop(['임신_성공_여부', 'ID'], axis=1)
TRJXFG_y = TRJXFG_train['임신_성공_여부']

TRVNRY_X = TRVNRY_train.drop(['임신_성공_여부', 'ID'], axis=1)
TRVNRY_y = TRVNRY_train['임신_성공_여부']

TRXQMD_X = TRXQMD_train.drop(['임신_성공_여부', 'ID'], axis=1)
TRXQMD_y = TRXQMD_train['임신_성공_여부']

TRYBLT_X = TRYBLT_train.drop(['임신_성공_여부', 'ID'], axis=1)
TRYBLT_y = TRYBLT_train['임신_성공_여부']

TRZKPL_X = TRZKPL_train.drop(['임신_성공_여부', 'ID'], axis=1)
TRZKPL_y = TRZKPL_train['임신_성공_여부']

### 인코딩 

In [5]:
categorical_columns = [
    "시술_당시_나이",
    "임신_시도_또는_마지막_임신_경과_연수",
    "시술_유형",
    "배란_유도_유형",
    "배아_생성_주요_이유",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이",
    "변환된_특정_시술_유형",
    "채취_해동_차이",
    "해동_혼합_차이",
    "혼합_이식_차이",
    "이식_해동_차이"
]

In [6]:
from sklearn.preprocessing import OrdinalEncoder

# 모든 범주형 변수를 문자열로 변환
TRCMWS_X[categorical_columns] = TRCMWS_X[categorical_columns].astype(str)
TRDQAZ_X[categorical_columns] = TRDQAZ_X[categorical_columns].astype(str)
TRJXFG_X[categorical_columns] = TRJXFG_X[categorical_columns].astype(str)
TRVNRY_X[categorical_columns] = TRVNRY_X[categorical_columns].astype(str)
TRXQMD_X[categorical_columns] = TRXQMD_X[categorical_columns].astype(str)
TRYBLT_X[categorical_columns] = TRYBLT_X[categorical_columns].astype(str)
TRZKPL_X[categorical_columns] = TRZKPL_X[categorical_columns].astype(str)

TRCMWS_test[categorical_columns] = TRCMWS_test[categorical_columns].astype(str)
TRDQAZ_test[categorical_columns] = TRDQAZ_test[categorical_columns].astype(str)
TRJXFG_test[categorical_columns] = TRJXFG_test[categorical_columns].astype(str)
TRVNRY_test[categorical_columns] = TRVNRY_test[categorical_columns].astype(str)
TRXQMD_test[categorical_columns] = TRXQMD_test[categorical_columns].astype(str)
TRYBLT_test[categorical_columns] = TRYBLT_test[categorical_columns].astype(str)
TRZKPL_test[categorical_columns] = TRZKPL_test[categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

TRCMWS_X[categorical_columns] = encoder.fit_transform(TRCMWS_X[categorical_columns])
TRDQAZ_X[categorical_columns] = encoder.fit_transform(TRDQAZ_X[categorical_columns])
TRJXFG_X[categorical_columns] = encoder.fit_transform(TRJXFG_X[categorical_columns])
TRVNRY_X[categorical_columns] = encoder.fit_transform(TRVNRY_X[categorical_columns])
TRXQMD_X[categorical_columns] = encoder.fit_transform(TRXQMD_X[categorical_columns])
TRYBLT_X[categorical_columns] = encoder.fit_transform(TRYBLT_X[categorical_columns])
TRZKPL_X[categorical_columns] = encoder.fit_transform(TRZKPL_X[categorical_columns])

TRCMWS_test[categorical_columns] = encoder.transform(TRCMWS_test[categorical_columns])
TRDQAZ_test[categorical_columns] = encoder.transform(TRDQAZ_test[categorical_columns])
TRJXFG_test[categorical_columns] = encoder.transform(TRJXFG_test[categorical_columns])
TRVNRY_test[categorical_columns] = encoder.transform(TRVNRY_test[categorical_columns])
TRXQMD_test[categorical_columns] = encoder.transform(TRXQMD_test[categorical_columns])
TRYBLT_test[categorical_columns] = encoder.transform(TRYBLT_test[categorical_columns])
TRZKPL_test[categorical_columns] = encoder.transform(TRZKPL_test[categorical_columns])

## Modeling

In [7]:
from sklearn.model_selection import train_test_split

# 데이터 분할
TRCMWS_X_train, TRCMWS_X_test, TRCMWS_y_train, TRCMWS_y_test = train_test_split(TRCMWS_X, TRCMWS_y, test_size=0.2, random_state=42)
TRDQAZ_X_train, TRDQAZ_X_test, TRDQAZ_y_train, TRDQAZ_y_test = train_test_split(TRDQAZ_X, TRDQAZ_y, test_size=0.2, random_state=42)
TRJXFG_X_train, TRJXFG_X_test, TRJXFG_y_train, TRJXFG_y_test = train_test_split(TRJXFG_X, TRJXFG_y, test_size=0.2, random_state=42)
TRVNRY_X_train, TRVNRY_X_test, TRVNRY_y_train, TRVNRY_y_test = train_test_split(TRVNRY_X, TRVNRY_y, test_size=0.2, random_state=42)
TRXQMD_X_train, TRXQMD_X_test, TRXQMD_y_train, TRXQMD_y_test = train_test_split(TRXQMD_X, TRXQMD_y, test_size=0.2, random_state=42)
TRYBLT_X_train, TRYBLT_X_test, TRYBLT_y_train, TRYBLT_y_test = train_test_split(TRYBLT_X, TRYBLT_y, test_size=0.2, random_state=42)
TRZKPL_X_train, TRZKPL_X_test, TRZKPL_y_train, TRZKPL_y_test = train_test_split(TRZKPL_X, TRZKPL_y, test_size=0.2, random_state=42)

TRXQMD_model

In [8]:
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 6000),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'max_depth': trial.suggest_int('max_depth', -1, 512),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.2, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'min_split_gain': trial.suggest_float('min_split_gain', 1e-8, 1.0, log=True),
        'random_state': 42,
        'boosting_type': 'gbdt',
        'verbose': -1
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(TRXQMD_X_train, TRXQMD_y_train)
    
    y_pred_proba = model.predict_proba(TRXQMD_X_test)[:, 1]
    
    auc = roc_auc_score(TRXQMD_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=800)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-10 00:22:02,350] A new study created in memory with name: no-name-54933175-34f6-4656-9948-3d65bce8a0fd
[I 2025-02-10 00:23:26,183] Trial 0 finished with value: 0.7135052269799488 and parameters: {'n_estimators': 3096, 'num_leaves': 176, 'max_depth': 379, 'learning_rate': 0.00039703052516526385, 'min_child_samples': 69, 'subsample': 0.33866105340579244, 'colsample_bytree': 0.6259879571438196, 'reg_alpha': 0.2434683235011865, 'reg_lambda': 0.0018539486809870257, 'min_split_gain': 0.0014975154840096672}. Best is trial 0 with value: 0.7135052269799488.
[I 2025-02-10 00:25:09,641] Trial 1 finished with value: 0.7105509015993143 and parameters: {'n_estimators': 3933, 'num_leaves': 467, 'max_depth': 193, 'learning_rate': 0.0001370013177486692, 'min_child_samples': 88, 'subsample': 0.48232421686662424, 'colsample_bytree': 0.9105728015988317, 'reg_alpha': 0.0008104721797936143, 'reg_lambda': 4.760461055030466e-05, 'min_split_gain': 0.006770396786602001}. Best is trial 0 with value: 0

Best trial:
  Value: 0.7188568914915668
  Params: 
    n_estimators: 5374
    num_leaves: 2
    max_depth: 149
    learning_rate: 0.027079336661169923
    min_child_samples: 44
    subsample: 0.8227361351552893
    colsample_bytree: 0.6590472451622268
    reg_alpha: 0.6568161002695931
    reg_lambda: 1.751055467841516e-08
    min_split_gain: 0.021509158668256725


.